In [3]:
import datetime as dt
from datetime import date
import os
import sys
import numpy as np
import pandas as pd
from glob import glob
import sqlalchemy
import pymysql
import time, datetime
from datetime import datetime, timedelta
from datetime import timedelta, date
import time
from numpy import sort

In [5]:
from numpy import unicode_


base = pd.read_csv(r"D:\analise_corr()\analise_dummy_ou_outros (version 1).csv",sep=';', encoding='ISO-8859-1')
base.head()


C:\Users\lucas.valdomiro\AppData\Local\Temp\ipykernel_17952\3712388123.py:4: DtypeWarning: Columns (90,91,96,98,99,101,102) have mixed types. Specify dtype option on import or set low_memory=False.
  base = pd.read_csv(r"D:\analise_corr()\analise_dummy_ou_outros (version 1).csv",sep=';', encoding='ISO-8859-1')


,customer_answer,created_at,tipo_pedido_nps,tipo_venda,pv_pai,Atraso_Transp_s,Flag Atraso Fornecedor_s,Flag Atraso Coleta_s,Flag Atraso Cliente_s,[FLAG]Cancelado_S,...,[[entrega]] Achei o prazo de entrega muito longo.],[[entrega]] Achei o valor do frete muito alto.],[[entrega]] Não foi possível agendar a entrega.],"[atendimento]] (aberta) Outro. Por favor, especifique:]",(aberta) Como foi sua experiência enquanto esperava o seu produto?,[compra]] (aberta) Foi fácil realizar sua compra?],"[entrega]] (aberta) Por favor, nos conte mais sobre a sua experiência.",[entrega]] (aberta) Você ficou satisfeito(a) com a entrega?,"[pagamento]] (aberta) Visando melhorar essa experiência, você pode contar quais d","[produto]] (aberta) Por favor, nos conte mais sobre a sua resposta"
0,10,18/10/2022 12:19,Mista,1,28162012,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10,10/10/2022 13:40,1P,1,28096157,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10,18/10/2022 13:11,1P,1,27804324,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10,06/10/2022 12:35,1P,1,28032244,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10,27/10/2022 15:11,1P,1,28214644,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
base['Flag Atraso Transporte2'].dtypes

dtype('float64')

In [16]:
import re


def somase(data_frame):

    colunas = ['Flag Atraso Transporte2','Flag Atraso Fornecedor3','Flag Atraso Coleta4','Flag Atraso Cliente5','[FLAG]Cancelado',
    '[FLAG]Recompra_ff','[FLAG]Erro prazo transporte validacao','[FLAG]Cross','[FLAG]Atraso_criacao_OC','[FLAG]Reprogramacao_OC',
    '[FLAG]Atraso Fornecedor','[FLAG] Entregue','[FLAG] Pratica_indevida','[FLAG] Sac_pos_entrega','[FLAG] Reversa','[FLAG] PJ',
    '[FLAG] Mais de uma entrega','[FLAG]Importacao WMS','[FLAG] Recompra','[FLAG] Entrega_Mista','[FLAG] Virada_Faturamento',
    'FLAG - Infos_encontradas','[FLAG] Atraso limite','[FLAG] Oc sem bip','[FLAG] Insucesso','[FLAG] End. N.Localizado/Ausente',
    '[FLAG] Tempo Habil','[FLAG] Atraso Redespacho','[FLAG] Diarias','[FLAG] Atraso Coletas','[FLAG] Leves','[FLAG] Atraso CD',
    '[FLAG] Recompra assistencia','[FLAG] Recompra Total','[FLAG] Origem Recompra','[FLAG] Atraso Cliente*','FLAG_Parceiro',
    'FLAG_terceiras_parceiras','[FLAG] - Detrator_mais_entrega','[FLAG] - Categorias_detrator','[FLAG] Private_Label','Guide_flag'
                ]

    for coluna in colunas:
        print('ta fazendo a coluna: ', coluna)
        nova_coluna = data_frame.groupby('pv_pai')[coluna].sum()
        data_frame = data_frame.merge(nova_coluna, left_on='pv_pai', right_on='pv_pai',how='left')

        data_frame.rename(columns={coluna + '_y': coluna + '_soma'}, inplace=True,errors='raise')


    return data_frame

somase(base)

base.head()




ta fazendo a coluna:  Flag Atraso Transporte2
ta fazendo a coluna:  Flag Atraso Fornecedor3
ta fazendo a coluna:  Flag Atraso Coleta4
ta fazendo a coluna:  Flag Atraso Cliente5
ta fazendo a coluna:  [FLAG]Cancelado
ta fazendo a coluna:  [FLAG]Recompra_ff
ta fazendo a coluna:  [FLAG]Erro prazo transporte validacao
ta fazendo a coluna:  [FLAG]Cross
ta fazendo a coluna:  [FLAG]Atraso_criacao_OC
ta fazendo a coluna:  [FLAG]Reprogramacao_OC
ta fazendo a coluna:  [FLAG]Atraso Fornecedor
ta fazendo a coluna:  [FLAG] Entregue
ta fazendo a coluna:  [FLAG] Pratica_indevida
ta fazendo a coluna:  [FLAG] Sac_pos_entrega
ta fazendo a coluna:  [FLAG] Reversa
ta fazendo a coluna:  [FLAG] PJ
ta fazendo a coluna:  [FLAG] Mais de uma entrega
ta fazendo a coluna:  [FLAG]Importacao WMS
ta fazendo a coluna:  [FLAG] Recompra
ta fazendo a coluna:  [FLAG] Entrega_Mista
ta fazendo a coluna:  [FLAG] Virada_Faturamento
ta fazendo a coluna:  FLAG - Infos_encontradas
ta fazendo a coluna:  [FLAG] Atraso limite
ta fa

,customer_answer,created_at,tipo_pedido_nps,tipo_venda,pv_pai,Atraso_Transp_s,Flag Atraso Fornecedor_s,Flag Atraso Coleta_s,Flag Atraso Cliente_s,[FLAG]Cancelado_S,...,[[entrega]] Achei o prazo de entrega muito longo.],[[entrega]] Achei o valor do frete muito alto.],[[entrega]] Não foi possível agendar a entrega.],"[atendimento]] (aberta) Outro. Por favor, especifique:]",(aberta) Como foi sua experiência enquanto esperava o seu produto?,[compra]] (aberta) Foi fácil realizar sua compra?],"[entrega]] (aberta) Por favor, nos conte mais sobre a sua experiência.",[entrega]] (aberta) Você ficou satisfeito(a) com a entrega?,"[pagamento]] (aberta) Visando melhorar essa experiência, você pode contar quais d","[produto]] (aberta) Por favor, nos conte mais sobre a sua resposta"
0,10,18/10/2022 12:19,Mista,1,28162012,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10,10/10/2022 13:40,1P,1,28096157,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10,18/10/2022 13:11,1P,1,27804324,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10,06/10/2022 12:35,1P,1,28032244,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10,27/10/2022 15:11,1P,1,28214644,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
